# Pyspark checkpoint 2 Data cleaning

contributions: RISHAB AGRAHARI
               SUBHAM SANKET
               

In [ ]:
import findspark
findspark.init('/usr/local/spark')

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, BooleanType, DateType

In [12]:
spark=SparkSession.builder.appName("retail_dataset").getOrCreate()

In [15]:
mydf = spark.read.csv('User_product_purchase_details_p2.csv',inferSchema = True, header = True)

In [16]:
mydf.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



In [17]:
mydf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c)
              for c in mydf.schema.names]).show()

+-------+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|      0|         0|            0|                         0|             0|                 0|            173638|            383247|       0|
+-------+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+



In [28]:
mydf = mydf.withColumn('new1', split(col('Product_Category_2'), ' ').getItem(0).cast('float')) \
        .withColumn('new2', split(col('Product_Category_3'), ' ').getItem(0).cast('float'))\
        
mydf = mydf.drop('Product_Category_2','Product_Category_3')
mydf = mydf.withColumnRenamed('new1', 'Product_Category_2') \
        .withColumnRenamed('new2', 'Product_Category_3') \
        

In [29]:
mydf.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Purchase: integer (nullable = true)
 |-- Product_Category_2: float (nullable = true)
 |-- Product_Category_3: float (nullable = true)



In [30]:
mydf.show(10)

+-------+----------+-------------+--------------------------+--------------+------------------+--------+------------------+------------------+
|User_ID|Product_ID|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Purchase|Product_Category_2|Product_Category_3|
+-------+----------+-------------+--------------------------+--------------+------------------+--------+------------------+------------------+
|1000001| P00069042|            A|                         2|             0|                 3|    8370|              null|              null|
|1000001| P00248942|            A|                         2|             0|                 1|   15200|               6.0|              14.0|
|1000001| P00087842|            A|                         2|             0|                12|    1422|              null|              null|
|1000001| P00085442|            A|                         2|             0|                12|    1057|              14.0|              null|

In [35]:
mydf.select('Product_Category_3').show(5)

+------------------+
|Product_Category_3|
+------------------+
|              null|
|              14.0|
|              null|
|              null|
|              null|
+------------------+
only showing top 5 rows



In [36]:
catCols = ['User_ID','Product_Category_1','Product_Category_2','Product_Category_3','Marital_Status']
mydf.select([countDistinct(c).alias(c)
              for c in catCols]).show()

+-------+------------------+------------------+------------------+--------------+
|User_ID|Product_Category_1|Product_Category_2|Product_Category_3|Marital_Status|
+-------+------------------+------------------+------------------+--------------+
|   5891|                20|                17|                15|             2|
+-------+------------------+------------------+------------------+--------------+

